In [87]:
import pandas as pd
import numpy as np
df=pd.read_csv('/content/merged_output.csv',low_memory=False)

In [88]:
'''
특이사항
난이도계수: 보통 nan 값이 많고 0에서 1로 갈수록 값의 빈도가 적어짐. nan은 기술평가 x 공사, 난이도 계수가 있으면 낙찰가가 더 높음-> 열을 지우지 말고 난이도계수 적용여부 + 난이도 계수 따로
관리비비율: 대부분 0이고 많아도 2%정도임 -> 일단 0이면 관련 제도가 적용 안 된 걸 수도 있으니 적용여부 +관리비비율 형태
예산대비추정가: 추정가비 -> inf 인경우(예산 값을 못받아왔음)도 있어서 이값이 유효한지 열추가로  전처리 진행함
관급비비중: 많으면 20% 정도 대부분 값 있음 -> 이상치 있고 0이 많아서 관급비 적용여부, 값 전처리 진행함
순공사비: 대부분 값 있음 음수인경우도 있음-> 일반적으로 공사비가 음수라고 돈을 주는 건 아닐 것임 오류일수도있고 실제 의미가 잇을 수도 있으니 음수여부 column추가, 기존 음수는 0으로 처리
VAT비율: 대부분 0.1 그런데 0,1 인경우도 있음 -> 0은 면세,비과세 등 케이스라고 보고 1은 세금이 있긴 한걸로 판단하여 전처리, 값이 너무 크면 0.1로 처리 갈수록 값의 빈도가 적어짐. nan은 기술평가 x 공사, 난이도 계수가 있으면 낙찰가가 더 높음-> 열을 지우지 말고 난이도계수 적용여부 + 난이도 계수 따로
관리비비율: 대부분 0이고 많아도 2%정도임 -> 일단 0이면 관련 제도가 적용 안 된 걸 수도 있으니 적용여부 +관리비비율 형태
예산대비추정가: 추정가비 -> inf 인경우(예산 값을 못받아왔음)도 있어서 이값이 유효한지 열추가로  전처리 진행함
관급비비중: 많으면 20% 정도 대부분 값 있음 -> 이상치 있고 0이 많아서 관급비 적용여부, 값 전처리 진행함
순공사비: 대부분 값 있음 음수인경우도 있음-> 일반적으로 공사비가 음수라고 돈을 주는 건 아닐 것임 오류일수도있고 실제 의미가 잇을 수도 있으니 음수여부 column추가, 기존 음수는 0으로 처리
VAT비율: 대부분 0.1 그런데 0,1 인경우도 있음 -> 0은 면세,비과세 등 케이스라고 보고 1은 세금이 있긴 한걸로 판단하여 전처리, 값이 너무 크면 0.1로 처리
각종기간: 기간들. 보통 몇일 안되지만 100일 넘는 장기 케이스도 있음->입찰준비기간은 짧을수록 준비 여유가 없을 것임. 그러므로 크릴핑 후 단기기간 열 추가. 공고-개찰 길수록 경쟁이 많긴하겠지만 낙찰가에 바로 영향은 안 줄 것임. 클리핑처리, 그대신 장기공고 유무열을 추가. 자격등록기간도 짧을수록 빡셀 것임.
참가제한:널값 x y,n임 -> 이진인코딩 진행
업종제한:yes거나 널이거나 -> api문제인지 제한이 없는지 알 수가 없음 drop하는걸
공사지역: 공사지역. -> 광역으로 묶음 세종도 있던데 특별자치시라 광역처럼 따로 구분함
가산점지역: 지역은 보통 공사장소랑 큰 차이 x 동일한 경우가 많음. 지역이 가까우면 엮이는 케이스는 종종 있음. 김해-부산 서울-경기 -> 공사지역도 있어서 가산점 지역 존재 여부로 묶어버림
지역의무공동계약여부: 널값 x -> 이진인코딩
재공고여부: 널값 x -> 이진인코딩

'''

'\n특이사항\n난이도계수: 보통 nan 값이 많고 0에서 1로 갈수록 값의 빈도가 적어짐. nan은 기술평가 x 공사, 난이도 계수가 있으면 낙찰가가 더 높음-> 열을 지우지 말고 난이도계수 적용여부 + 난이도 계수 따로\n관리비비율: 대부분 0이고 많아도 2%정도임 -> 일단 0이면 관련 제도가 적용 안 된 걸 수도 있으니 적용여부 +관리비비율 형태\n예산대비추정가: 추정가비 -> inf 인경우(예산 값을 못받아왔음)도 있어서 이값이 유효한지 열추가로  전처리 진행함\n관급비비중: 많으면 20% 정도 대부분 값 있음 -> 이상치 있고 0이 많아서 관급비 적용여부, 값 전처리 진행함\n순공사비: 대부분 값 있음 음수인경우도 있음-> 일반적으로 공사비가 음수라고 돈을 주는 건 아닐 것임 오류일수도있고 실제 의미가 잇을 수도 있으니 음수여부 column추가, 기존 음수는 0으로 처리\nVAT비율: 대부분 0.1 그런데 0,1 인경우도 있음 -> 0은 면세,비과세 등 케이스라고 보고 1은 세금이 있긴 한걸로 판단하여 전처리, 값이 너무 크면 0.1로 처리 갈수록 값의 빈도가 적어짐. nan은 기술평가 x 공사, 난이도 계수가 있으면 낙찰가가 더 높음-> 열을 지우지 말고 난이도계수 적용여부 + 난이도 계수 따로\n관리비비율: 대부분 0이고 많아도 2%정도임 -> 일단 0이면 관련 제도가 적용 안 된 걸 수도 있으니 적용여부 +관리비비율 형태\n예산대비추정가: 추정가비 -> inf 인경우(예산 값을 못받아왔음)도 있어서 이값이 유효한지 열추가로  전처리 진행함\n관급비비중: 많으면 20% 정도 대부분 값 있음 -> 이상치 있고 0이 많아서 관급비 적용여부, 값 전처리 진행함\n순공사비: 대부분 값 있음 음수인경우도 있음-> 일반적으로 공사비가 음수라고 돈을 주는 건 아닐 것임 오류일수도있고 실제 의미가 잇을 수도 있으니 음수여부 column추가, 기존 음수는 0으로 처리\nVAT비율: 대부분 0.1 그런데 0,1 인경우도 있음 -> 0은 면세,비과세 등 케이스라고 

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646584 entries, 0 to 646583
Data columns (total 28 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   공고번호        646584 non-null  object 
 1   차수          646584 non-null  int64  
 2   공고명_x       646584 non-null  object 
 3   기초금액        646584 non-null  int64  
 4   예가범위        646584 non-null  float64
 5   난이도계수       16932 non-null   float64
 6   안전관리비비율     646584 non-null  float64
 7   품질관리비비율     646584 non-null  float64
 8   공고명_y       626701 non-null  object 
 9   추정가격        626700 non-null  float64
 10  낙찰하한율       626434 non-null  float64
 11  예산대비추정가     626699 non-null  float64
 12  관급비비중       626699 non-null  float64
 13  순공사비        582190 non-null  float64
 14  VAT비율       582189 non-null  float64
 15  입찰준비기간      626701 non-null  float64
 16  공고개찰기간      626701 non-null  float64
 17  자격등록기간      626701 non-null  float64
 18  참가제한        626701 non-null  object 
 19  업종

In [90]:
df1=df.dropna(subset=['낙찰가'])

In [91]:
print(df1.info())

<class 'pandas.core.frame.DataFrame'>
Index: 597810 entries, 1 to 646583
Data columns (total 28 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   공고번호        597810 non-null  object 
 1   차수          597810 non-null  int64  
 2   공고명_x       597810 non-null  object 
 3   기초금액        597810 non-null  int64  
 4   예가범위        597810 non-null  float64
 5   난이도계수       11815 non-null   float64
 6   안전관리비비율     597810 non-null  float64
 7   품질관리비비율     597810 non-null  float64
 8   공고명_y       597810 non-null  object 
 9   추정가격        597809 non-null  float64
 10  낙찰하한율       597587 non-null  float64
 11  예산대비추정가     597808 non-null  float64
 12  관급비비중       597808 non-null  float64
 13  순공사비        554670 non-null  float64
 14  VAT비율       554669 non-null  float64
 15  입찰준비기간      597810 non-null  float64
 16  공고개찰기간      597810 non-null  float64
 17  자격등록기간      597810 non-null  float64
 18  참가제한        597810 non-null  object 
 19  업종제한   

In [93]:
df1['낙찰가'].describe()

,낙찰가
count,5.978100e+05
mean,3.049332e+08
std,2.590376e+09
min,4.112600e+02
25%,4.414902e+07
50%,8.593754e+07
75%,1.931652e+08
max,6.329280e+11


In [94]:
#확인해보면 낙찰'가'인데 최솟값이 400 경우가 있음 단위가 차이있을 수 있으나 불러왔을 때 이상치에 속해서 제거 필요

In [95]:
ratio = df1["낙찰가"] / df1["기초금액"]
ratio.describe()

,0
count,597810.000000
mean,0.883407
std,0.015373
min,0.087108
25%,0.875990
50%,0.881435
75%,0.888122
max,1.016312


In [96]:
# 상위 0.1% 제거
upper_limit = df1["낙찰가"].quantile(0.999)
# 하위 1,000,000 제거
df1 = df1[(df1["낙찰가"] <= upper_limit) & (df1["낙찰가"] >= 1_000_000)]

In [97]:
#낙찰가가 너무 크거나 낙찰가 자체가 너무 낮으면 제거

In [98]:
ratio = df1["낙찰가"] / df1["기초금액"]
ratio.describe()

,0
count,595890.000000
mean,0.883433
std,0.015247
min,0.087108
25%,0.876000
50%,0.881444
75%,0.888126
max,1.016312


In [99]:
#전반적으로 낙찰가 값 일부를 제거한 후 낙찰가 분포 값들이 조금 더 커짐

In [100]:
df1.groupby("난이도계수")["낙찰가"].describe()

,count,mean,std,min,25%,50%,75%,max
난이도계수,,,,,,,,
0.00,7193.0,1.172701e+09,1.794234e+09,5938542.0,2.959786e+08,5.990184e+08,1.334028e+09,1.664248e+10
0.01,56.0,1.783793e+08,2.319468e+08,15976476.0,3.834139e+07,5.738322e+07,2.359580e+08,9.973610e+08
0.04,1.0,2.130860e+08,NaN,213086000.0,2.130860e+08,2.130860e+08,2.130860e+08,2.130860e+08
0.05,5.0,1.521599e+08,6.487650e+07,63546715.0,1.026122e+08,1.929578e+08,1.929578e+08,2.087252e+08
0.10,2.0,1.763339e+08,4.706918e+07,143051000.0,1.596925e+08,1.763339e+08,1.929754e+08,2.096169e+08
0.20,7.0,6.860649e+07,6.919266e+07,17503445.0,2.141892e+07,3.730288e+07,1.050228e+08,1.725557e+08
0.22,3.0,1.336352e+08,8.367814e+07,77320337.0,8.555827e+07,9.379620e+07,1.617926e+08,2.297890e+08
0.25,1.0,3.394550e+07,NaN,33945500.0,3.394550e+07,3.394550e+07,3.394550e+07,3.394550e+07
0.30,2.0,3.495572e+07,0.000000e+00,34955723.0,3.495572e+07,3.495572e+07,3.495572e+07,3.495572e+07


In [101]:
#값들이 대부분 0, nan이도 많음-> nan이 난이도계수가 적용이 안되는 공고인지, 누락인지 확실히 모르기에 난이도계수 적용여부 추가

In [102]:
df1["난이도계수_적용여부"] = df1["난이도계수"].notna().astype(int)
df1["난이도계수"] = df1["난이도계수"].fillna(0)

df1.groupby("난이도계수_적용여부")["낙찰가"].describe()

,count,mean,std,min,25%,50%,75%,max
난이도계수_적용여부,,,,,,,,
0,584626.0,2.043460e+08,4.150276e+08,1000100.0,43894355.0,84372430.5,1.843219e+08,1.566754e+10
1,11264.0,2.973001e+09,3.643324e+09,1962000.0,343255075.0,981081560.0,5.225149e+09,1.664248e+10


In [103]:
#적용여부에 따라 분포 차이 有

In [104]:
df1[["안전관리비비율","품질관리비비율"]].describe()

,안전관리비비율,품질관리비비율
count,595890.000000,595890.000000
mean,0.010555,0.000464
std,0.009265,0.003274
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.012996,0.000000
75%,0.018159,0.000000
max,0.281407,0.621201


In [105]:
#관리리 비율들이 대부분 0임 -> 관리비가 필요 없는 공고 있을 수 있을 수도 있어서 관리비 적용 여부 추가

In [106]:
df1["안전관리비_적용여부"] = (df1["안전관리비비율"] > 0).astype(int)
df1["품질관리비_적용여부"] = (df1["품질관리비비율"] > 0).astype(int)

In [107]:
df1.groupby(["안전관리비_적용여부","품질관리비_적용여부"])["낙찰가"].describe()

count          mean           std         min  \
안전관리비_적용여부 품질관리비_적용여부                                                     
0          0           224437.0  9.097713e+07  2.048053e+08   1000100.0   
           1              136.0  7.195017e+08  1.047571e+09  33061710.0   
1          0           328932.0  2.346559e+08  4.563256e+08   1001440.0   
           1            42385.0  1.303563e+09  2.194181e+09   1071100.0   

                                25%          50%           75%           max  
안전관리비_적용여부 품질관리비_적용여부                                                         
0          0           3.521551e+07   60563030.0  1.029267e+08  1.566754e+10  
           1           1.871558e+08  448801354.0  8.210858e+08  7.594350e+09  
1          0           5.061909e+07  102904119.0  2.356401e+08  1.617153e+10  
           1           2.194247e+08  443587500.0  1.291898e+09  1.664248e+10

In [108]:
#적용여부에 따른 낙찰가 차이가 있음

In [109]:
df1["예산대비추정가"].describe()

,예산대비추정가
count,5.958880e+05
mean,inf
std,NaN
min,9.090909e-12
25%,7.070848e-01
50%,9.090909e-01
75%,9.090909e-01
max,inf


In [110]:
#예산에 0인 부분이 있어서 값이 inf로 나옴. 최소값이 너무 작음

In [111]:
df1["예산대비추정가"].replace([np.inf, -np.inf], np.nan, inplace=True)
df1["예산대비추정가_유효"] = df1["예산대비추정가"].notna().astype(int)
df1["예산대비추정가"] = df1["예산대비추정가"].fillna(1.0)
df1["예산대비추정가"] = df1["예산대비추정가"].clip(0.3, 1.5)

/tmp/ipython-input-1785437448.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1["예산대비추정가"].replace([np.inf, -np.inf], np.nan, inplace=True)


In [112]:
#예산대비 추정가 중 nan 여부 확인하게 하고 값이 너무 작은 경우 값을 0.3으로 고정하게함

In [113]:
df1["예산대비추정가"].describe()

,예산대비추정가
count,595890.000000
mean,0.797573
std,0.159775
min,0.300000
25%,0.707085
50%,0.909091
75%,0.909091
max,1.500000


In [114]:
df1['관급비비중'].describe()

,관급비비중
count,595888.000000
mean,0.152627
std,0.932137
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,178.917198


In [115]:
#관급비 비중이 대부분 0임. 관급비가 적용안되는 공고일 수 있어서 적용여부 컬럼 추가. 또한 비중인데 1을 넘는 값이 있어서 값 클리핑 시행

In [116]:
df1["관급비_적용여부"] = (df1["관급비비중"] > 0).astype(int)
df1["관급비비중"] = df1["관급비비중"].clip(0, 1.0)

In [117]:
df1['관급비비중'].describe()

,관급비비중
count,595888.00000
mean,0.08614
std,0.23358
min,0.00000
25%,0.00000
50%,0.00000
75%,0.00000
max,1.00000


In [118]:
df1.groupby("관급비_적용여부")["낙찰가"].describe()

,count,mean,std,min,25%,50%,75%,max
관급비_적용여부,,,,,,,,
0,467142.0,1.666627e+08,4.104294e+08,1000100.0,40724051.25,75369000.0,1.561036e+08,1.664013e+10
1,128748.0,5.833001e+08,1.361636e+09,1071100.0,74995369.50,172773532.0,4.564787e+08,1.664248e+10


In [119]:
#관급비 적용 여부에 따른 낙찰가 차이 존재

In [120]:
df1['순공사비'].describe()

,순공사비
count,5.529670e+05
mean,2.057775e+08
std,6.740371e+08
min,-6.565532e+10
25%,3.629700e+07
50%,7.311764e+07
75%,1.630800e+08
max,1.967232e+10


In [121]:
#순공사비에 -가 있음. 데이터 불러오는 중 문제인 건지 알 수 없어서 음수여부 컬럼 추가

In [122]:
df1["순공사비_음수여부"] = (df1["순공사비"] < 0).astype(int)
df1.loc[df1["순공사비"] <= 0, "순공사비"] = np.nan

In [123]:
df1.groupby("순공사비_음수여부")["낙찰가"].describe()

,count,mean,std,min,25%,50%,75%,max
순공사비_음수여부,,,,,,,,
0,571973.0,2.596395e+08,7.595102e+08,1000100.0,44358500.0,86295152.0,193988000.0,1.664248e+10
1,23917.0,1.859373e+08,4.500870e+08,2063114.0,42829561.0,79086251.0,168644600.0,1.539957e+10


In [124]:
#순공사비 음수여부에 따른 낙찰가 값 차이 존재

In [125]:
df1['VAT비율'].describe()

,VAT비율
count,552966.000000
mean,0.100071
std,0.024785
min,0.000000
25%,0.100000
50%,0.100000
75%,0.100000
max,1.000000


In [126]:
#VAT비율 경우 대부분 값이 0.1로 보이나 1인 경우도 존재. 세금이 100%일 수 없기에 이상치 처리 진행+ 세금이 0%면 비과세 같은 경우일수도 있기에 세금 적용여부 커럶 추가

In [127]:
df1["VAT_적용여부"] = (df1["VAT비율"] > 0).astype(int)
df1.loc[df1["VAT비율"] > 0.2, "VAT비율"] = np.nan
df1["VAT비율"] = df1["VAT비율"].fillna(0.1)

In [128]:
#VAT 비율이 0.2보다 크면 음수 처리 후 0.1로 처리

In [129]:
df1.groupby("VAT_적용여부")["낙찰가"].describe()

,count,mean,std,min,25%,50%,75%,max
VAT_적용여부,,,,,,,,
0,45379.0,2.202609e+08,6.254187e+08,1024273.0,38908705.5,74191000.0,168710936.0,1.644454e+10
1,550511.0,2.596835e+08,7.589543e+08,1000100.0,44825370.0,86882100.0,195250645.0,1.664248e+10


In [130]:
#적용여부에 따라 낙찰가 값이 큰 차이는 없긴한데 어느정도 있음

In [131]:
df1[['입찰준비기간','공고개찰기간','자격등록기간']].describe()

,입찰준비기간,공고개찰기간,자격등록기간
count,595890.000000,595890.000000,595890.000000
mean,1.773260,6.332760,5.557128
std,2.315370,2.384644,2.395067
min,0.000000,0.110509,0.000069
25%,0.487141,5.699850,4.959401
50%,0.912448,5.916389,5.164005
75%,2.679546,6.848229,6.094997
max,53.809734,55.908785,55.809734


In [132]:
#기간이 1일 이내인 경우도 있고 50일처럼 긴 경우도 있어서 입찰준비기간, 자격등록기간은 짧을수록 준비시간이 부족해 참여에 힘들고, 공고개찰기간이 길수록 참여하기 쉬워 따로 열 추가

In [133]:
upper = df1["입찰준비기간"].quantile(0.99)
df1["입찰준비기간"] = df1["입찰준비기간"].clip(upper=upper)
df1["초단기_입찰준비"] = (df1["입찰준비기간"] < 0.5).astype(int)

upper = df1["공고개찰기간"].quantile(0.99)
df1["공고개찰기간"] = df1["공고개찰기간"].clip(upper=upper)
df1["장기공고"] = (df1["공고개찰기간"] >= 30).astype(int)

upper = df1["자격등록기간"].quantile(0.99)
df1["자격등록기간"] = df1["자격등록기간"].clip(upper=upper)
df1["자격등록_촉박"] = (df1["자격등록기간"] < 3).astype(int)

In [134]:
df1.groupby(["초단기_입찰준비","장기공고","자격등록_촉박"])["낙찰가"].describe()

count          mean           std        min  \
초단기_입찰준비 장기공고 자격등록_촉박                                                    
0        0    0        425317.0  2.822531e+08  8.294535e+08  1004000.0   
              1         20457.0  9.627619e+07  2.555038e+08  1011891.0   
1        0    0        138022.0  2.155827e+08  5.307920e+08  1000100.0   
              1         12094.0  9.774466e+07  2.466657e+08  1034490.0   

                              25%         50%          75%           max  
초단기_입찰준비 장기공고 자격등록_촉박                                                     
0        0    0        45610000.0  88314346.0  208440055.0  1.664248e+10  
              1        36140000.0  61408800.0  106101590.0  1.495641e+10  
1        0    0        43277300.0  83586691.5  184399714.0  1.617153e+10  
              1        37332875.0  62556164.5  103538631.5  1.397148e+10

In [135]:
#입찰관련 기간에 따라 낙찰가 분포 차이가 있음

In [136]:
df1["참가제한"] = (df1["참가제한"] == "Y").astype(int)

In [137]:
df1.groupby("참가제한")["낙찰가"].describe()

,count,mean,std,min,25%,50%,75%,max
참가제한,,,,,,,,
0,595117.0,2.565674e+08,7.498648e+08,1000100.0,44292400.0,86013950.0,192904281.0,1.664248e+10
1,773.0,3.444511e+08,5.988690e+08,7271000.0,65680583.0,146960000.0,360640078.0,6.993800e+09


In [138]:
#참가제한에 따라 낙찰가 차이가 있음

In [139]:
df1["업종제한"].describe()

,업종제한
count,589464
unique,2
top,Y
freq,587595


In [140]:
#업종 제한의 경우 결측값이 꽤 많은데도 값이 null이나 Y라서 데이터상 오류인지, N을 null로 처리한지 알 수가 없음-> 열 제거

In [141]:
df1.drop(columns=["업종제한"], inplace=True)

In [142]:
df1["공사지역"].value_counts()

,count
공사지역,
서울특별시,11468
부산광역시,9274
광주광역시,6844
충청북도 청주시,6316
대구광역시,6271
...,...
경기도 부천시 소사구,150
경기도 부천시 오정구,102
국외소재,17


In [143]:
#지역이 많아서 광역시로 나눌 예정. 세종의 경우 특별자치시라 정책 같은 것의 차이가 있을 수 있어 따로 추가 분리

In [144]:
def map_region_wide(region):
    if pd.isna(region):
        return "기타"

    # 광역자치단체 키워드 매핑
    region_map = {
        "서울": "서울",
        "부산": "부산",
        "대구": "대구",
        "인천": "인천",
        "광주": "광주",
        "대전": "대전",
        "울산": "울산",
        "세종": "세종",
        "경기": "경기",
        "강원": "강원",
        "충청북도": "충북",
        "충청남도": "충남",
        "전라북도": "전북",
        "전라남도": "전남",
        "경상북도": "경북",
        "경상남도": "경남",
        "제주": "제주"
    }

    # 국외 처리
    if "국외" in region:
        return "기타"

    # 키워드 포함 여부로 판별
    for key, value in region_map.items():
        if key in region:
            return value

    # 어디에도 안 걸리면 기타
    return "기타"

In [145]:
df1["공사지역_광역"] = df1["공사지역"].apply(map_region_wide)

In [146]:
df1["공사지역_광역"].value_counts()

,count
공사지역_광역,
경기,91175
경북,72210
경남,58884
전남,57728
충남,49132
강원,43758
충북,37862
서울,36414
전북,23348


In [147]:
df1.groupby("공사지역_광역")["낙찰가"].describe()

,count,mean,std,min,25%,50%,75%,max
공사지역_광역,,,,,,,,
강원,43758.0,2.478782e+08,7.164336e+08,1020535.0,44965499.75,82768249.0,1.747653e+08,1.664013e+10
경기,91175.0,2.528594e+08,7.817658e+08,1000100.0,52717522.50,92468300.0,1.930229e+08,1.663810e+10
경남,58884.0,2.214354e+08,6.723521e+08,1007500.0,41168725.00,76167000.0,1.675598e+08,1.661490e+10
경북,72210.0,2.224737e+08,7.354754e+08,1183000.0,39429000.00,70794400.0,1.594758e+08,1.612555e+10
광주,15629.0,2.087098e+08,6.480965e+08,1010530.0,39129832.00,79502800.0,1.737549e+08,1.593978e+10
기타,21499.0,2.452896e+08,7.121505e+08,1050650.0,39493437.50,75982670.0,1.816365e+08,1.634603e+10
대구,14443.0,3.096294e+08,7.796328e+08,1492973.0,62612195.00,126951000.0,2.565534e+08,1.611221e+10
대전,12280.0,3.084972e+08,8.433111e+08,4453940.0,52260500.00,114214700.0,2.618858e+08,1.664248e+10
부산,20482.0,3.157406e+08,7.795696e+08,1114680.0,60462947.50,122449364.5,2.738981e+08,1.658194e+10


In [148]:
#수도권쪽이 낙찰가가 확실히 비쌈

In [149]:
df1 = pd.get_dummies(
    df1,
    columns=["공사지역_광역"],
    drop_first=True
)
df1.drop(columns=["공사지역"])

,공고번호,차수,공고명_x,기초금액,예가범위,난이도계수,안전관리비비율,품질관리비비율,공고명_y,추정가격,...,공사지역_광역_부산,공사지역_광역_서울,공사지역_광역_세종,공사지역_광역_울산,공사지역_광역_인천,공사지역_광역_전남,공사지역_광역_전북,공사지역_광역_제주,공사지역_광역_충남,공사지역_광역_충북
1,R25BK00961669,0,우치공원 체육시설(테니스장 노후시설 등) 보수공사(건축공사),225710000,3.0,0.0,0.020884,0.000000,우치공원 체육시설(테니스장 노후시설 등) 보수공사(건축공사),2.051909e+08,...,False,False,False,False,False,False,False,False,False,False
2,R25BK00959124,0,경기도장애인복지종합지원센터 2025년 장애인의 삶의 질 향상을 위한 맞춤형 생활환경...,421404000,3.0,0.0,0.019651,0.000000,경기도장애인복지종합지원센터 2025년 장애인의 삶의 질 향상을 위한 맞춤형 생활환경...,3.830945e+08,...,False,False,False,False,False,False,False,False,False,False
4,R25BK00958710,0,전주중앙중 식생활관 리모델링 통신공사,30724000,3.0,0.0,0.018135,0.000000,전주중앙중 식생활관 리모델링 통신공사,2.793091e+07,...,False,False,False,False,False,False,False,False,False,False
11,R25BK00955401,0,청계초 영어교실 환경개선공사,77090000,3.0,0.0,0.018885,0.000000,청계초 영어교실 환경개선공사,7.008182e+07,...,False,False,False,False,False,False,False,False,False,False
13,R25BK00961727,0,해봄센터 구축사업 공간 재구조화 공사(통신),46002000,3.0,0.0,0.017512,0.000000,해봄센터 구축사업 공간 재구조화 공사(통신),4.182000e+07,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646579,20230930294,0,덕정교차로~온산로간(대3-20호선) 확장공사,2643640000,3.0,0.0,0.009980,0.004622,덕정교차로~온산로간(대3-20호선) 확장공사,2.403309e+09,...,False,False,False,True,False,False,False,False,False,False
646580,20231000031,0,잠원빗물펌프장 노후시설물 정비공사,662330000,3.0,0.0,0.019350,0.000755,잠원빗물펌프장 노후시설물 정비공사,6.021182e+08,...,False,True,False,False,False,False,False,False,False,False
646581,20231000033,0,반포동 737-19 주변 사각형거 보수공사,615840000,3.0,0.0,0.017010,0.000858,반포동 737-19 주변 사각형거 보수공사,5.598545e+08,...,False,True,False,False,False,False,False,False,False,False
646582,20231000034,0,사임당로39~서초대로50길41 주변 사각형거 보수공사,868043000,3.0,0.0,0.016896,0.000723,사임당로39~서초대로50길41 주변 사각형거 보수공사,7.891300e+08,...,False,True,False,False,False,False,False,False,False,False


In [150]:
df1['가산점지역1'].value_counts()

,count
가산점지역1,
서울특별시,496
경기도,489
경상남도,265
경상북도,212
부산광역시,159
충청남도,145
전북특별자치도,134
충청북도,116
대구광역시,107


In [151]:
gasan_cols = ["가산점지역1", "가산점지역2", "가산점지역3", "가산점지역4"]

df1["가산점지역_적용여부"] = (
    df1[gasan_cols].notna().any(axis=1).astype(int)
)

df1.drop(columns=gasan_cols, inplace=True)

In [152]:
#가산점 지역이 4가지 지역까지 되는데 보통 공사지역과 비슷해서 가산점지역이 있는지 여부로 압축함

In [153]:
df1.groupby("가산점지역_적용여부")["낙찰가"].describe()

,count,mean,std,min,25%,50%,75%,max
가산점지역_적용여부,,,,,,,,
0,593162.0,2.556763e+08,7.426733e+08,1000100.0,44271781.5,85973000.0,192856395.0,1.664013e+10
1,2728.0,4.752207e+08,1.671387e+09,2597830.0,57165870.0,113556000.0,238009280.0,1.664248e+10


In [154]:
#가산점 적용여부에 따라 낙찰가 값이 있는 편

In [155]:
df1['지역의무공동계약여부'].value_counts()

,count
지역의무공동계약여부,
N,588756
Y,7134


In [156]:
df1["지역의무공동계약여부"] = (df1["지역의무공동계약여부"] == "Y").astype(int)

In [157]:
df1.groupby("지역의무공동계약여부")["낙찰가"].describe()

,count,mean,std,min,25%,50%,75%,max
지역의무공동계약여부,,,,,,,,
0,588756.0,2.243111e+08,5.860764e+08,1000100.0,4.400281e+07,8.484381e+07,1.864542e+08,1.664248e+10
1,7134.0,2.928142e+09,3.373058e+09,3652299.0,1.189045e+09,1.579208e+09,2.559478e+09,1.664013e+10


In [158]:
#지역의무공동계약여부에 따라 낙찰가에 차이가 있음

In [159]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 595890 entries, 1 to 646583
Data columns (total 51 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   공고번호        595890 non-null  object 
 1   차수          595890 non-null  int64  
 2   공고명_x       595890 non-null  object 
 3   기초금액        595890 non-null  int64  
 4   예가범위        595890 non-null  float64
 5   난이도계수       595890 non-null  float64
 6   안전관리비비율     595890 non-null  float64
 7   품질관리비비율     595890 non-null  float64
 8   공고명_y       595890 non-null  object 
 9   추정가격        595889 non-null  float64
 10  낙찰하한율       595755 non-null  float64
 11  예산대비추정가     595890 non-null  float64
 12  관급비비중       595888 non-null  float64
 13  순공사비        528835 non-null  float64
 14  VAT비율       595890 non-null  float64
 15  입찰준비기간      595890 non-null  float64
 16  공고개찰기간      595890 non-null  float64
 17  자격등록기간      595890 non-null  float64
 18  참가제한        595890 non-null  int64  
 19  공사지역   

In [160]:
df1["재공고여부"] = (df1["재공고여부"] == "Y").astype(int)
df1.groupby("재공고여부")["낙찰가"].describe()

,count,mean,std,min,25%,50%,75%,max
재공고여부,,,,,,,,
0,593259.0,2.552533e+08,7.425248e+08,1000100.0,44267050.0,85952930.0,192701490.0,1.664248e+10
1,2631.0,5.786825e+08,1.694563e+09,1046930.0,58898885.0,123140800.0,306551080.0,1.626738e+10


In [161]:
#재공고여부에 따른 낙찰가 차이 있음

In [162]:
diff_mask = df1["공고명_x"] != df1["공고명_y"]

diff_count = diff_mask.sum()
diff_count

np.int64(0)

In [163]:
df1 = df1.drop(columns=["공고명_y"])
df1 = df1.rename(columns={"공고명_x": "공고명"})

In [164]:
df1.columns

Index(['공고번호', '차수', '공고명', '기초금액', '예가범위', '난이도계수', '안전관리비비율', '품질관리비비율',
       '추정가격', '낙찰하한율', '예산대비추정가', '관급비비중', '순공사비', 'VAT비율', '입찰준비기간',
       '공고개찰기간', '자격등록기간', '참가제한', '공사지역', '지역의무공동계약여부', '재공고여부', '낙찰가',
       '난이도계수_적용여부', '안전관리비_적용여부', '품질관리비_적용여부', '예산대비추정가_유효', '관급비_적용여부',
       '순공사비_음수여부', 'VAT_적용여부', '초단기_입찰준비', '장기공고', '자격등록_촉박', '공사지역_광역_경기',
       '공사지역_광역_경남', '공사지역_광역_경북', '공사지역_광역_광주', '공사지역_광역_기타', '공사지역_광역_대구',
       '공사지역_광역_대전', '공사지역_광역_부산', '공사지역_광역_서울', '공사지역_광역_세종', '공사지역_광역_울산',
       '공사지역_광역_인천', '공사지역_광역_전남', '공사지역_광역_전북', '공사지역_광역_제주', '공사지역_광역_충남',
       '공사지역_광역_충북', '가산점지역_적용여부'],
      dtype='object')

In [165]:
df1=df1.drop(columns=["공고번호","차수","공고명","공사지역"])

In [166]:
len(df1.columns)

46

In [167]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 595890 entries, 1 to 646583
Data columns (total 46 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   기초금액        595890 non-null  int64  
 1   예가범위        595890 non-null  float64
 2   난이도계수       595890 non-null  float64
 3   안전관리비비율     595890 non-null  float64
 4   품질관리비비율     595890 non-null  float64
 5   추정가격        595889 non-null  float64
 6   낙찰하한율       595755 non-null  float64
 7   예산대비추정가     595890 non-null  float64
 8   관급비비중       595888 non-null  float64
 9   순공사비        528835 non-null  float64
 10  VAT비율       595890 non-null  float64
 11  입찰준비기간      595890 non-null  float64
 12  공고개찰기간      595890 non-null  float64
 13  자격등록기간      595890 non-null  float64
 14  참가제한        595890 non-null  int64  
 15  지역의무공동계약여부  595890 non-null  int64  
 16  재공고여부       595890 non-null  int64  
 17  낙찰가         595890 non-null  float64
 18  난이도계수_적용여부  595890 non-null  int64  
 19  안전관리비_적

In [168]:
#추정가, 낙찰하한율, 관급비 비중은 null 값이 적어서 열 자체 제거, 순공사비는 결측치가 많아서 결측 여부 적용
df1 = df1.dropna(subset=["추정가격", "낙찰하한율","관급비비중"])
df1["순공사비_결측여부"] = df1["순공사비"].isna().astype(int)
df1["순공사비"] = df1["순공사비"].fillna(0)

In [169]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 595753 entries, 1 to 646583
Data columns (total 47 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   기초금액        595753 non-null  int64  
 1   예가범위        595753 non-null  float64
 2   난이도계수       595753 non-null  float64
 3   안전관리비비율     595753 non-null  float64
 4   품질관리비비율     595753 non-null  float64
 5   추정가격        595753 non-null  float64
 6   낙찰하한율       595753 non-null  float64
 7   예산대비추정가     595753 non-null  float64
 8   관급비비중       595753 non-null  float64
 9   순공사비        595753 non-null  float64
 10  VAT비율       595753 non-null  float64
 11  입찰준비기간      595753 non-null  float64
 12  공고개찰기간      595753 non-null  float64
 13  자격등록기간      595753 non-null  float64
 14  참가제한        595753 non-null  int64  
 15  지역의무공동계약여부  595753 non-null  int64  
 16  재공고여부       595753 non-null  int64  
 17  낙찰가         595753 non-null  float64
 18  난이도계수_적용여부  595753 non-null  int64  
 19  안전관리비_적

In [170]:
df1.to_csv("dataset.csv",index=False,encoding="utf-8-sig")

In [171]:
#잘 읽히는지 테스트
df2=pd.read_csv("dataset.csv")

In [172]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595753 entries, 0 to 595752
Data columns (total 47 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   기초금액        595753 non-null  int64  
 1   예가범위        595753 non-null  float64
 2   난이도계수       595753 non-null  float64
 3   안전관리비비율     595753 non-null  float64
 4   품질관리비비율     595753 non-null  float64
 5   추정가격        595753 non-null  float64
 6   낙찰하한율       595753 non-null  float64
 7   예산대비추정가     595753 non-null  float64
 8   관급비비중       595753 non-null  float64
 9   순공사비        595753 non-null  float64
 10  VAT비율       595753 non-null  float64
 11  입찰준비기간      595753 non-null  float64
 12  공고개찰기간      595753 non-null  float64
 13  자격등록기간      595753 non-null  float64
 14  참가제한        595753 non-null  int64  
 15  지역의무공동계약여부  595753 non-null  int64  
 16  재공고여부       595753 non-null  int64  
 17  낙찰가         595753 non-null  float64
 18  난이도계수_적용여부  595753 non-null  int64  
 19  안전